In [64]:
"""
Notebook: 01_data_download.ipynb
MIMIC-III Demo Dataset Download and Initial Exploration
"""
# Cell 1: Setup and Imports
import pandas as pd
import numpy as np
import os
from pathlib import Path
import requests
import gzip
import shutil
from tqdm import tqdm
import json

# Load config
with open('../configs/config.json', 'r') as f:
    config = json.load(f)

print("✓ Imports complete")
print(f"Project: {config['project']['name']} v{config['project']['version']}")


✓ Imports complete
Project: TRANCE v1.0.0


In [65]:
# Cell 2: Download MIMIC-III Demo Files
"""
MIMIC-III Demo Dataset
- Publicly available (no PhysioNet credentials needed)
- Contains 100 patients
- Perfect for prototyping
"""

import requests
from tqdm import tqdm
from pathlib import Path

def download_file(url, destination):
    """Download a CSV file with progress bar and error handling"""
    try:
        response = requests.get(url, stream=True)
        if response.status_code != 200:
            print(f"⚠️ Failed to download {url} (HTTP {response.status_code})")
            return False

        total_size = int(response.headers.get('content-length', 0))
        with open(destination, 'wb') as f, tqdm(
            desc=destination.name,
            total=total_size if total_size > 0 else None,
            unit='iB',
            unit_scale=True,
            unit_divisor=1024,
        ) as pbar:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
                    pbar.update(len(chunk))

        # Verify that file is not HTML (in case of bad URL)
        with open(destination, 'r', encoding='utf-8', errors='ignore') as f:
            start = f.read(15)
            if "<html>" in start.lower():
                print(f"⚠️ Skipping {destination.name}: HTML page instead of CSV")
                destination.unlink(missing_ok=True)
                return False

        return True

    except Exception as e:
        print(f"❌ Error downloading {url}: {e}")
        return False


# ✅ Correct PhysioNet base URL (uses /files/, not /content/)
base_url = "https://physionet.org/files/mimiciii-demo/1.4/"

# Key tables we need
tables = [
    'ADMISSIONS.csv',
    'PATIENTS.csv',
    'DIAGNOSES_ICD.csv',
    'PROCEDURES_ICD.csv',
    'PRESCRIPTIONS.csv',
    'LABEVENTS.csv',
    'CHARTEVENTS.csv',
    'NOTEEVENTS.csv',
    'ICUSTAYS.csv',
    'D_ICD_DIAGNOSES.csv',
    'D_ICD_PROCEDURES.csv',
    'D_LABITEMS.csv'
]

# Create data directory
data_dir = Path('../data/raw')
data_dir.mkdir(parents=True, exist_ok=True)

print("⬇️ Downloading MIMIC-III Demo CSV files...\n")

for table in tables:
    url = base_url + table
    dest = data_dir / table

    if dest.exists():
        print(f"✓ {table} already exists")
        continue

    print(f"Downloading {table}...")
    success = download_file(url, dest)
    if success:
        print(f"✓ Downloaded {table}")
    else:
        print(f"❌ Failed to download {table}")

print("\n✅ All available CSV files downloaded successfully!")


⬇️ Downloading MIMIC-III Demo CSV files...

✓ ADMISSIONS.csv already exists
✓ PATIENTS.csv already exists
✓ DIAGNOSES_ICD.csv already exists
✓ PROCEDURES_ICD.csv already exists
✓ PRESCRIPTIONS.csv already exists
✓ LABEVENTS.csv already exists
✓ CHARTEVENTS.csv already exists
✓ NOTEEVENTS.csv already exists
✓ ICUSTAYS.csv already exists
✓ D_ICD_DIAGNOSES.csv already exists
✓ D_ICD_PROCEDURES.csv already exists
✓ D_LABITEMS.csv already exists

✅ All available CSV files downloaded successfully!


In [66]:
# # Cell 3: Extract CSV files
# """
# Extract gzipped files
# """
# print("Extracting CSV files...")

# for table in tables:
#     gz_file = data_dir / table
#     csv_file = data_dir / table.replace('.gz', '')
    
#     if csv_file.exists():
#         print(f"✓ {csv_file.name} already exists")
#     else:
#         print(f"Extracting {table}...")
#         with gzip.open(gz_file, 'rb') as f_in:
#             with open(csv_file, 'wb') as f_out:
#                 shutil.copyfileobj(f_in, f_out)
#         print(f"✓ Extracted {csv_file.name}")

# print("\n✅ All files extracted!")


In [67]:
# Cell 4: Load and explore key tables
"""
Initial Data Exploration
"""
print("Loading key tables...")
from pathlib import Path

data_dir = Path("../data/raw")  # if your notebook is in notebooks/

# Load tables
admissions = pd.read_csv(data_dir / 'ADMISSIONS.csv')
patients = pd.read_csv(data_dir / 'PATIENTS.csv')
diagnoses = pd.read_csv(data_dir / 'DIAGNOSES_ICD.csv')
notes = pd.read_csv(data_dir / 'NOTEEVENTS.csv')
labs = pd.read_csv(data_dir / 'LABEVENTS.csv')
prescriptions = pd.read_csv(data_dir / 'PRESCRIPTIONS.csv')

# Standardize column names to uppercase
admissions.columns = admissions.columns.str.upper()
patients.columns = patients.columns.str.upper()
diagnoses.columns = diagnoses.columns.str.upper()
notes.columns = notes.columns.str.upper()
labs.columns = labs.columns.str.upper()
prescriptions.columns = prescriptions.columns.str.upper()


print("✓ Tables loaded\n")

# Display table sizes
print("=" * 60)
print("MIMIC-III Demo Dataset Overview")
print("=" * 60)
print(f"Patients:        {len(patients):>10,} rows")
print(f"Admissions:      {len(admissions):>10,} rows")
print(f"Diagnoses:       {len(diagnoses):>10,} rows")
print(f"Clinical Notes:  {len(notes):>10,} rows")
print(f"Lab Events:      {len(labs):>10,} rows")
print(f"Prescriptions:   {len(prescriptions):>10,} rows")
print("=" * 60)


Loading key tables...
✓ Tables loaded

MIMIC-III Demo Dataset Overview
Patients:               100 rows
Admissions:             129 rows
Diagnoses:            1,761 rows
Clinical Notes:           0 rows
Lab Events:          76,074 rows
Prescriptions:       10,398 rows


In [68]:
# Cell 5: Admissions Table Exploration
"""
Explore ADMISSIONS table - our primary table
"""
print("ADMISSIONS Table Sample:")
print(admissions.head())
print("\nColumns:", admissions.columns.tolist())
print("\nData Types:")
print(admissions.dtypes)
print("\nMissing Values:")
print(admissions.isnull().sum())

# admissions.columns = admissions.columns.str.upper()

admissions['ADMITTIME'] = pd.to_datetime(admissions['ADMITTIME'])
admissions['DISCHTIME'] = pd.to_datetime(admissions['DISCHTIME'])
admissions['DEATHTIME'] = pd.to_datetime(admissions['DEATHTIME'])

admissions['LOS_DAYS'] = (admissions['DISCHTIME'] - admissions['ADMITTIME']).dt.total_seconds() / 86400

print("\nAdmission Statistics:")
print(admissions[['LOS_DAYS']].describe())


ADMISSIONS Table Sample:
   ROW_ID  SUBJECT_ID  HADM_ID            ADMITTIME            DISCHTIME  \
0   12258       10006   142345  2164-10-23 21:09:00  2164-11-01 17:15:00   
1   12263       10011   105331  2126-08-14 22:32:00  2126-08-28 18:59:00   
2   12265       10013   165520  2125-10-04 23:36:00  2125-10-07 15:13:00   
3   12269       10017   199207  2149-05-26 17:19:00  2149-06-03 18:42:00   
4   12270       10019   177759  2163-05-14 20:43:00  2163-05-15 12:00:00   

             DEATHTIME ADMISSION_TYPE         ADMISSION_LOCATION  \
0                  NaN      EMERGENCY       EMERGENCY ROOM ADMIT   
1  2126-08-28 18:59:00      EMERGENCY  TRANSFER FROM HOSP/EXTRAM   
2  2125-10-07 15:13:00      EMERGENCY  TRANSFER FROM HOSP/EXTRAM   
3                  NaN      EMERGENCY       EMERGENCY ROOM ADMIT   
4  2163-05-15 12:00:00      EMERGENCY  TRANSFER FROM HOSP/EXTRAM   

  DISCHARGE_LOCATION INSURANCE LANGUAGE  RELIGION MARITAL_STATUS  \
0   HOME HEALTH CARE  Medicare      NaN  

In [69]:
# Cell 6: Patient Demographics
"""
Explore patient demographics
"""
# Merge patients with admissions for analysis
patient_admissions = admissions.merge(patients, on='SUBJECT_ID')

# Calculate age at admission
patient_admissions['AGE'] = patient_admissions['ADMITTIME'].dt.year - patient_admissions['DOB'].apply(lambda x: pd.to_datetime(x).year)

print("Patient Demographics:")
print("\nGender Distribution:")
print(patient_admissions['GENDER'].value_counts())

print("\nAge Statistics:")
print(patient_admissions['AGE'].describe())

print("\nEthnicity Distribution:")
print(patient_admissions['ETHNICITY'].value_counts().head(10))

print("\nInsurance Distribution:")
print(patient_admissions['INSURANCE'].value_counts())

Patient Demographics:

Gender Distribution:
GENDER
M    70
F    59
Name: count, dtype: int64

Age Statistics:
count    129.000000
mean      84.922481
std       61.115619
min       17.000000
25%       64.000000
50%       73.000000
75%       83.000000
max      300.000000
Name: AGE, dtype: float64

Ethnicity Distribution:
ETHNICITY
WHITE                                                       86
HISPANIC/LATINO - PUERTO RICAN                              15
UNKNOWN/NOT SPECIFIED                                       11
BLACK/AFRICAN AMERICAN                                       7
OTHER                                                        3
ASIAN                                                        2
HISPANIC OR LATINO                                           2
AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE     2
UNABLE TO OBTAIN                                             1
Name: count, dtype: int64

Insurance Distribution:
INSURANCE
Medicare      98
Private       24
Medicai

In [70]:
import pandas as pd
import numpy as np
from pathlib import Path

data_dir = Path("../data/raw")  # Adjust if your notebook is elsewhere

# Load tables
admissions = pd.read_csv(data_dir / 'ADMISSIONS.csv')
diagnoses = pd.read_csv(data_dir / 'DIAGNOSES_ICD.csv')
noteevents = pd.read_csv(data_dir / 'NOTEEVENTS.csv')
procedures = pd.read_csv(data_dir / 'PROCEDURES_ICD.csv')
patients = pd.read_csv(data_dir / 'PATIENTS.csv')
prescriptions = pd.read_csv(data_dir / 'PRESCRIPTIONS.csv')

# Capitalize column names
admissions.columns = admissions.columns.str.upper()
diagnoses.columns = diagnoses.columns.str.upper()
noteevents.columns = noteevents.columns.str.upper()
procedures.columns = procedures.columns.str.upper()
patients.columns = patients.columns.str.upper()
prescriptions.columns = prescriptions.columns.str.upper()

# Merge admissions with patient info
admissions_patients = admissions.merge(patients, on='SUBJECT_ID', suffixes=('_ADM', '_PAT'))

# Convert to datetime
admissions_patients['ADMITTIME'] = pd.to_datetime(admissions_patients['ADMITTIME'])
admissions_patients['DISCHTIME'] = pd.to_datetime(admissions_patients['DISCHTIME'])
admissions_patients['DOB'] = pd.to_datetime(admissions_patients['DOB'])

# Calculate age and LOS
admissions_patients['AGE'] = admissions_patients['ADMITTIME'].dt.year - admissions_patients['DOB'].dt.year
admissions_patients['LOS_DAYS'] = (admissions_patients['DISCHTIME'] - admissions_patients['ADMITTIME']).dt.total_seconds() / (24*3600)

# Updated discharge text generator
def generate_discharge_text_updated(hadm_id):
    row = admissions_patients[admissions_patients['HADM_ID'] == hadm_id].iloc[0]

    age = row.get('AGE', 'unknown age')
    gender = row.get('GENDER', 'unknown gender')
    if gender == 'M':
        gender = 'male'
    elif gender == 'F':
        gender = 'female'
    admit_date = row.get('ADMITTIME').strftime('%Y-%m-%d') if pd.notna(row.get('ADMITTIME')) else 'unknown date'
    los_days = round(row.get('LOS_DAYS', 0), 1)
    admission_type = row.get('ADMISSION_TYPE', 'unspecified')
    discharge_location = row.get('DISCHARGE_LOCATION', 'unknown')

    # Diagnoses
    patient_diag = diagnoses[diagnoses['HADM_ID'] == hadm_id]
    diag_text = ", ".join(patient_diag['ICD9_CODE'].astype(str).tolist()) or "No significant diagnoses recorded"

    # Procedures
    patient_proc = procedures[procedures['HADM_ID'] == hadm_id]
    proc_text = ", ".join(patient_proc['ICD9_CODE'].astype(str).tolist()) or "No procedures performed"

    # Prescriptions
    patient_rx = prescriptions[prescriptions['HADM_ID'] == hadm_id]
    rx_list = []
    for _, rx in patient_rx.iterrows():
        med = rx.get('DRUG', 'Unknown drug')
        dose = rx.get('DOSE_VAL_RX', '')
        unit = rx.get('DOSE_UNIT_RX', '')
        route = rx.get('ROUTE', '')
        parts = [str(part) for part in [dose, unit, route, med] if part]
        rx_list.append(" ".join(parts))
    rx_text = "; ".join(rx_list) if rx_list else "No medications prescribed"

    discharge_summary = (
        f"The patient is a {age}-year-old {gender} admitted on {admit_date} "
        f"via {admission_type}. During the {los_days}-day hospital stay, "
        f"the following diagnoses were recorded: {diag_text}. "
        f"Procedures performed include: {proc_text}. "
        f"At discharge, the patient was prescribed: {rx_text}. "
        f"The patient was discharged to {discharge_location} with appropriate follow-up instructions."
    )

    return discharge_summary

# Populate NOTEEVENTS
for idx, row in admissions_patients.iterrows():
    hadm_id = row['HADM_ID']

    # Find existing notes
    note_idx = noteevents[noteevents['HADM_ID'] == hadm_id].index

    if len(note_idx) == 0:
        new_row = {
            'ROW_ID': noteevents['ROW_ID'].max() + 1 if len(noteevents) > 0 else 1,
            'SUBJECT_ID': row['SUBJECT_ID'],
            'HADM_ID': hadm_id,
            'CHARTDATE': row['ADMITTIME'].strftime('%Y-%m-%d') if pd.notna(row['ADMITTIME']) else '',
            'CHARTTIME': row['ADMITTIME'].strftime('%H:%M:%S') if pd.notna(row['ADMITTIME']) else '',
            'STORETIME': row['ADMITTIME'] if pd.notna(row['ADMITTIME']) else '',
            'CATEGORY': 'Discharge summary',
            'DESCRIPTION': 'Synthetic discharge summary',
            'CGID': np.nan,
            'ISERROR': False,
            'TEXT': generate_discharge_text_updated(hadm_id)
        }
        noteevents = pd.concat([noteevents, pd.DataFrame([new_row])], ignore_index=True)
    else:
        # Update empty TEXT fields
        for i in note_idx:
            if pd.isna(noteevents.at[i, 'TEXT']) or noteevents.at[i, 'TEXT'].strip() == '':
                noteevents.at[i, 'TEXT'] = generate_discharge_text_updated(hadm_id)
                noteevents.at[i, 'CATEGORY'] = 'Discharge summary'
                noteevents.at[i, 'DESCRIPTION'] = 'Synthetic discharge summary'

# Save updated NOTEEVENTS
noteevents.to_csv(data_dir / 'NOTEEVENTS_with_discharge_text.csv', index=False)

# Preview
print(noteevents[['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'CATEGORY', 'DESCRIPTION', 'TEXT']].head(2).to_string())


  ROW_ID SUBJECT_ID HADM_ID           CATEGORY                  DESCRIPTION                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [71]:
# Cell 7: Clinical Notes Analysis
"""
Explore clinical notes - critical for embeddings
"""
print("Clinical Notes Analysis")
print("=" * 60)

print("\nNote Categories:")
print(noteevents['CATEGORY'].value_counts())

print("\nNote Descriptions:")
print(noteevents['DESCRIPTION'].value_counts().head(10))

# Filter discharge summaries
discharge_notes = noteevents[noteevents['CATEGORY'] == 'Discharge summary']
print(f"\nDischarge Summaries: {len(discharge_notes)} notes")

# Sample discharge summary
print("\nSample Discharge Summary:")
print("=" * 60)
if len(discharge_notes) > 0:
    sample = discharge_notes.iloc[0]
    print(f"Subject ID: {sample['SUBJECT_ID']}")
    print(f"Admission ID: {sample['HADM_ID']}")
    print(f"Chart Date: {sample['CHARTDATE']}")
    print(f"\nText (first 500 chars):")
    print(sample['TEXT'][:500] if pd.notna(sample['TEXT']) else "No text available")
    print("...")

Clinical Notes Analysis

Note Categories:
CATEGORY
Discharge summary    129
Name: count, dtype: int64

Note Descriptions:
DESCRIPTION
Synthetic discharge summary    129
Name: count, dtype: int64

Discharge Summaries: 129 notes

Sample Discharge Summary:
Subject ID: 10006
Admission ID: 142345
Chart Date: 2164-10-23

Text (first 500 chars):
The patient is a 70-year-old female admitted on 2164-10-23 via EMERGENCY. During the 8.8-day hospital stay, the following diagnoses were recorded: 99591, 99662, 5672, 40391, 42731, 4280, 4241, 4240, 2874, 03819, 7850, E8791, V090, 56211, 28529, 25000, V5867, E9342, 41401, 2749, 3051. Procedures performed include: 9749, 5491, 3895, 3995, 3893, 9907, 14. At discharge, the patient was prescribed: 3 ml IV Sodium Chloride 0.9%  Flush; 10 mg PO Glipizide; 150 mg PO Metoprolol; 800 mg PO Sevelamer; 0 U
...


In [72]:
# Cell 8: Readmission Analysis
"""
Identify readmissions in demo data
"""
print("Readmission Analysis")
print("=" * 60)

# Ensure datetime
admissions['ADMITTIME'] = pd.to_datetime(admissions['ADMITTIME'], errors='coerce')
admissions['DISCHTIME'] = pd.to_datetime(admissions['DISCHTIME'], errors='coerce')

# Drop rows where dates are missing
admissions_clean = admissions.dropna(subset=['ADMITTIME', 'DISCHTIME'])

# Sort by patient and admission time
admissions_sorted = admissions_clean.sort_values(['SUBJECT_ID', 'ADMITTIME'])

# Calculate time to next admission
admissions_sorted['NEXT_ADMIT'] = admissions_sorted.groupby('SUBJECT_ID')['ADMITTIME'].shift(-1)
admissions_sorted['DAYS_TO_NEXT_ADMIT'] = (
    (admissions_sorted['NEXT_ADMIT'] - admissions_sorted['DISCHTIME']).dt.total_seconds() / 86400
)

# Flag 30-day readmission
admissions_sorted['READMIT_30'] = (admissions_sorted['DAYS_TO_NEXT_ADMIT'] <= 30) & (admissions_sorted['DAYS_TO_NEXT_ADMIT'] > 0)

print(f"Total admissions: {len(admissions_sorted)}")
print(f"Patients with multiple admissions: {admissions_sorted.groupby('SUBJECT_ID').size().gt(1).sum()}")
print(f"30-day readmissions: {admissions_sorted['READMIT_30'].sum()} ({admissions_sorted['READMIT_30'].mean()*100:.1f}%)")



Readmission Analysis
Total admissions: 129
Patients with multiple admissions: 14
30-day readmissions: 11 (8.5%)


In [73]:
# Cell 9: Diagnosis Analysis
"""
Analyze diagnosis patterns
"""
print("Diagnosis Analysis")
print("=" * 60)

# Load diagnosis descriptions
d_icd = pd.read_csv(data_dir / 'D_ICD_DIAGNOSES.csv')
d_icd.columns = d_icd.columns.str.upper()

# Merge with diagnoses
diagnoses_with_desc = diagnoses.merge(d_icd, on=['ICD9_CODE'], how='left')

print(f"Total diagnosis records: {len(diagnoses)}")
print(f"Unique ICD9 codes: {diagnoses['ICD9_CODE'].nunique()}")

print("\nTop 10 Most Common Diagnoses:")
top_diagnoses = diagnoses_with_desc.groupby(['ICD9_CODE', 'LONG_TITLE']).size().sort_values(ascending=False).head(10)
for (code, title), count in top_diagnoses.items():
    print(f"{code:>6} | {count:>3} | {title}")


Diagnosis Analysis
Total diagnosis records: 1761
Unique ICD9 codes: 581

Top 10 Most Common Diagnoses:
  4019 |  53 | Unspecified essential hypertension
 42731 |  48 | Atrial fibrillation
  5849 |  45 | Acute kidney failure, unspecified
  4280 |  39 | Congestive heart failure, unspecified
 25000 |  31 | Diabetes mellitus without mention of complication, type II or unspecified type, not stated as uncontrolled
 51881 |  31 | Acute respiratory failure
  2724 |  29 | Other and unspecified hyperlipidemia
  5990 |  27 | Urinary tract infection, site not specified
   486 |  26 | Pneumonia, organism unspecified
  2859 |  25 | Anemia, unspecified


In [74]:
# Cell 10: Save Exploration Summary
"""
Save exploration results for reference
"""
admissions['ADMITTIME'] = pd.to_datetime(admissions['ADMITTIME'], errors='coerce')
admissions['DISCHTIME'] = pd.to_datetime(admissions['DISCHTIME'], errors='coerce')

admissions['LOS_DAYS'] = (admissions['DISCHTIME'] - admissions['ADMITTIME']).dt.total_seconds() / (24*3600)

summary = {
    "dataset": "MIMIC-III Demo",
    "version": "1.4",
    "download_date": pd.Timestamp.now().strftime("%Y-%m-%d"),
    "statistics": {
        "n_patients": int(len(patients)),
        "n_admissions": int(len(admissions)),
        "n_diagnoses": int(len(diagnoses)),
        "n_notes": int(len(notes)),
        "n_discharge_summaries": int(len(discharge_notes)),
        "readmission_rate_30d": float(admissions_sorted['READMIT_30'].mean()),
        "avg_los_days": float(admissions['LOS_DAYS'].mean())  # now exists
    },
    "data_quality": {
        "admissions_missing_dischtime": int(admissions['DISCHTIME'].isnull().sum()),
        "notes_missing_text": int(notes['TEXT'].isnull().sum()),
        "patients_missing_dob": int(patients['DOB'].isnull().sum())
    }
}


summary_path = Path('../outputs/results/data_exploration_summary.json')
summary_path.parent.mkdir(parents=True, exist_ok=True)

with open(summary_path, 'w') as f:
    json.dump(summary, f, indent=2)

print("✅ Exploration summary saved to:", summary_path)
print("\n" + "=" * 60)
print("Data Download and Exploration Complete!")
print("=" * 60)
print("\nNext Steps:")
print("1. Review the summary statistics above")
print("2. Proceed to notebook 02_cohort_definition.ipynb")
print("3. Define inclusion/exclusion criteria")

✅ Exploration summary saved to: ..\outputs\results\data_exploration_summary.json

Data Download and Exploration Complete!

Next Steps:
1. Review the summary statistics above
2. Proceed to notebook 02_cohort_definition.ipynb
3. Define inclusion/exclusion criteria
